# Illustration of MABS (Multivariate finite Alphabet Blind Separation) 


The MABS algorithm, which is analyzed in 

[1]  *Minimax estimation in linear models with unknown design over finite alphabets*, Behr M and Munk A, 	arXiv:1711.04145

takes as input a given finite alphabet $\mathfrak{A} = \{ a_1, \ldots, a_k\} \subset \mathbb{R}$, a given number of sources $m$, and a data matrix $Y$. 

It yields as output a factorization $Y \approx F \Omega$, where $F$ is an $n \times m$ source matrix which only takes value in the given finite alphabet, that is, $F_{ij}\in \mathfrak{A}$ for $i = 1, \ldots, n, j = 1,\ldots, m$, and $\Omega$ is a $m \times M$ weight matrix with $\Omega_{ij} \in \mathbb{R}_+$ and $\sum_{i = 1}^m \Omega_{ij} = 1$ for $i = 1, \ldots, m, j = 1, \ldots, M$.

In this github repository we provide an implementation of the algorithm, as well as all source code which generates figures and simulation results presented in [1]. For details on the algorithm and a theoretical analysis, we refer to [1]. In the following we provide a short illustration on how to use the implementation.

The implementation requires the R package *combinat*.

In [2]:
if (!require("combinat")) install.packages("combinat")

Loading required package: combinat


Attaching package: ‘combinat’


The following object is masked from ‘package:utils’:

    combn




First, we load the functions which provide the MABS implementation.

In [3]:
source("scripts/functions.R")

For illustration purposes, we generate a data matrix $Y$ with known source and weight matrix. First, we define the number of sources $m$, the standard deviation of the noise $\sigma$, the alphabet, the number of mixtures $M$, as well as the number of samples $n$.

In [44]:
m <- 3 #number of sources
sigma <- 0.5 #standard deviation of noise
al <- 0:2 #alphabet
M <- 30 #number of mixtures
n <- 500 #number of samples

We can generate a random $m \times M$ weight matrix $\Omega$ using the function *rOmega*. We also randomly select a source matrix, with elements uniformly selected from the alphabet.

In [51]:
trOm <- rOmega(m , M)
trF <- matrix(sample(al, n * m, replace = T), ncol = m)

For the data matrix $Y$ we add i.i.d. Gaussian noise to the mixture matrix $F \Omega$.

In [52]:
y <- trF %*% trOm + rnorm(n * M, sd = sigma)

Given the data matrix $Y$, the number of sources $m$, and the alphabet $\mathfrak{A}$ we can now estimate the weight matrix with the MABS algorithm as follows.

In [53]:
estOm <- estOmega(y, m, al)

str(estOm)

 num [1:3, 1:30] 0.401 0.385 0.524 0.418 0.303 ...
 - attr(*, "baseline")= num [1:30] 0.1644 -0.1963 -0.1303 -0.3257 0.0114 ...


Given the estimated $\hat{\Omega}$, the source matrix is then estimated row-wise, according to the closest mixture value.

In [54]:
Am <- as.matrix(expand.grid(rep(list(al), m))) #matrix with different alphabet combinations
valEst <- Am %*% estOm #matrix with different mixtures values

# estimate the finite alphabet source matrix row wise
indEstF <- sapply(1:nrow(y), function(i) which.min(colSums( (y[i,] - t(valEst) )^2 ) ))
estF <- Am[indEstF, ]

str(estF)

 int [1:500, 1:3] 2 1 2 2 1 0 1 1 2 1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:3] "Var1" "Var2" "Var3"


Weight and source matrix in the MABS model are only defined up to a permutation of the columns of $F$ and the rows of $\Omega$. In order to evaluate the quality of the estimates $\hat{F}$ and $\hat{\Omega}$ we can search through all permutations to find the one which fits best.

In [55]:
perm <- permn(1:m)
accA_perm <- numeric(length(perm))
    for(i in 1:length(perm)){
        accA_perm[i] <- mean(estF[, perm[[i]]] == trF)
      }

print(accA_perm)

perm_opt = perm[[which.max(accA_perm)]]
print(perm_opt)

estF <- estF[, perm_opt]
estOm <- estOm[perm_opt,]

[1] 0.5346667 0.3753333 0.4913333 0.8426667 0.5573333 0.3653333
[1] 3 2 1


Now we can compare the estimated weight and source matrix with the underlying truth.

In [56]:
print("Top columns of estimated weight matrix:")
print(head(t(estOm)))

print("Top columns of true weight matrix:")
print(head(t(trOm)))

print("Top rows of estimated source matrix:")
print(head(estF))

print("Top rows of true source matrix:")
print(head(trF))

[1] "Top columns of estimated weight matrix:"
           [,1]      [,2]        [,3]
[1,] 0.52360171 0.3851794  0.40122365
[2,] 0.39806026 0.3032816  0.41757281
[3,] 0.25533051 0.7027353  0.19437430
[4,] 0.01271637 0.3104841  0.48580015
[5,] 0.72221046 0.4019897 -0.05259798
[6,] 0.47759169 0.4069763  0.06147404
[1] "Top columns of true weight matrix:"
            [,1]       [,2]       [,3]
[1,] 0.397666742 0.24232133 0.36001193
[2,] 0.530189358 0.08325727 0.38655337
[3,] 0.236055095 0.61881709 0.14512781
[4,] 0.005615828 0.39801693 0.59636724
[5,] 0.671644026 0.24886919 0.07948678
[6,] 0.415061198 0.52858389 0.05635492
[1] "Top rows of estimated source matrix:"
     Var3 Var2 Var1
[1,]    0    0    2
[2,]    1    1    1
[3,]    0    1    2
[4,]    0    1    2
[5,]    2    0    1
[6,]    0    0    0
[1] "Top rows of true source matrix:"
     [,1] [,2] [,3]
[1,]    0    0    2
[2,]    1    1    1
[3,]    0    1    2
[4,]    0    1    2
[5,]    1    0    2
[6,]    0    0    0
